## T2

In [ ]:
import sys, re
sys.path.append('../../code/python')
import numpy as np
import pandas as pd
import helpers as hh
import matplotlib.pyplot as plt
from matplotlib.figure import SubplotParams
from matplotlib import cm, colors
%matplotlib inline

D = pd.HDFStore('../../data/tables/station_data.h5')
T = hh.stationize(D['ta_c'].xs('prom', 1, 'aggr').drop(10, 1, 'elev'))
sta = D['sta']
S = pd.HDFStore('../../data/tables/LinearLinear.h5')
Tm = S['T2']
Z = S['z']
Map = hh.basemap()

In [ ]:
Tm.items

In [ ]:
Z

In [ ]:
z = re.compile('d03_\d')
fig = plt.figure(
    figsize=(8, 8),
    subplotpars=SubplotParams(left=0.08, right=0.86, wspace=0.06, hspace=0.04))
for k in range(3):
    for j, x in enumerate(Tm.items):
        ax = plt.subplot(3, Tm.shape[0], k * Tm.shape[0] + j + 1)

        B = (Tm[x] - T).dropna(0, 'all')
        b = B.mean().dropna()
        dz = sta['elev'] - Z['d03_op' if z.search(x) else x]
        if k == 0:
            ax.set_title(x)
            D = pd.concat(
                (sta.loc[b.index, ('lon', 'lat')], dz), axis=1).dropna()
            m = 1750
        elif k == 1:
            D = pd.concat((sta.loc[:, ('lon', 'lat')], b), axis=1).dropna()
            m = 13
        else:
            D = pd.concat(
                (sta.loc[:, ('lon', 'lat')], b - 0.0065 * dz), axis=1).dropna()
        sm = cm.ScalarMappable(norm=colors.Normalize(vmin=-m, vmax=m))
        sm.set_array(D.iloc[:, -1])
        sm.set_cmap('coolwarm')
        for i, a in D.iterrows():
            Map.plot(
                a['lon'],
                a['lat'],
                'o',
                color=sm.to_rgba(a.iloc[-1]),
                latlon=True)
        Map.drawcoastlines()
        Map.drawparallels(range(-32, -28, 1), labels=[max(Tm.shape[0] - 2 - j, 0), 0, 0, 0])
        Map.drawmeridians(range(-72, -69, 1), labels=[0, 0, 0, max(k - 1, 0)])

        if j == 3:
            bb = ax.get_position()
            plt.colorbar(
                sm,
                cax=fig.add_axes([bb.x1 + 0.02, bb.y0, 0.02, bb.y1 - bb.y0]))

